# Day 1: Trading System Architecture Patterns

## Week 22: System Design for Quantitative Trading

**Learning Objectives:**
- Understand core trading system components and their interactions
- Implement event-driven architecture for trading systems
- Design modular, testable, and scalable trading infrastructure
- Build production-ready patterns for quant trading systems

**Architecture Overview:**
```
┌─────────────────────────────────────────────────────────────────────────┐
│                     TRADING SYSTEM ARCHITECTURE                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐               │
│  │ Market Data  │───▶│   Strategy   │───▶│    Order     │               │
│  │   Handler    │    │   Engine     │    │  Management  │               │
│  └──────────────┘    └──────────────┘    └──────────────┘               │
│         │                   │                   │                        │
│         ▼                   ▼                   ▼                        │
│  ┌──────────────────────────────────────────────────────┐               │
│  │                    EVENT BUS                          │               │
│  └──────────────────────────────────────────────────────┘               │
│         │                   │                   │                        │
│         ▼                   ▼                   ▼                        │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐               │
│  │    Risk      │    │  Portfolio   │    │  Execution   │               │
│  │   Manager    │    │   Manager    │    │   Handler    │               │
│  └──────────────┘    └──────────────┘    └──────────────┘               │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

**Key Design Principles:**
1. **Separation of Concerns** - Each component has a single responsibility
2. **Loose Coupling** - Components communicate through events/interfaces
3. **High Cohesion** - Related functionality grouped together
4. **Testability** - Each component can be tested in isolation

## 1. Import Required Libraries

Core libraries for building a trading system architecture:

In [1]:
# Core Python Libraries for System Design
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from enum import Enum, auto
from typing import (
    Dict, List, Optional, Callable, Any, 
    Protocol, TypeVar, Generic, Iterator
)
from collections import deque
from datetime import datetime, timedelta
from decimal import Decimal
import uuid
import logging
import time
import heapq
from queue import Queue, PriorityQueue
from threading import Lock, Event as ThreadEvent
import json

# Numerical and Data Libraries
import numpy as np
import pandas as pd

# Configure logging for system components
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Type variables for generic components
T = TypeVar('T')
EventType = TypeVar('EventType', bound='Event')

print("✅ All libraries imported successfully")
print("🏗️ Ready to build trading system architecture")

✅ All libraries imported successfully
🏗️ Ready to build trading system architecture


## 2. Event-Driven Architecture Pattern

Event-driven architecture is the backbone of modern trading systems. Components communicate through events rather than direct method calls, enabling:
- **Decoupling**: Components don't need to know about each other
- **Scalability**: Easy to add new event handlers
- **Testability**: Events can be replayed for testing
- **Auditability**: Full event log for debugging

### 2.1 Event Base Classes and Types

```
Event Flow in Trading System:
───────────────────────────────────────────────────────────────
MARKET_DATA_EVENT ──▶ SIGNAL_EVENT ──▶ ORDER_EVENT ──▶ FILL_EVENT
       │                    │               │              │
       ▼                    ▼               ▼              ▼
   DataHandler         Strategy        RiskManager   ExecutionHandler
                                           │
                                           ▼
                                       Portfolio
```

In [2]:
# ============================================================================
# EVENT TYPES ENUMERATION
# ============================================================================

class EventType(Enum):
    """Types of events in the trading system."""
    MARKET_DATA = auto()    # New market data received
    SIGNAL = auto()         # Trading signal generated
    ORDER = auto()          # Order to be executed
    FILL = auto()           # Order fill confirmation
    PORTFOLIO = auto()      # Portfolio update
    RISK = auto()           # Risk alert
    SYSTEM = auto()         # System-level events


class OrderSide(Enum):
    """Order direction."""
    BUY = "BUY"
    SELL = "SELL"


class OrderType(Enum):
    """Types of orders."""
    MARKET = "MARKET"
    LIMIT = "LIMIT"
    STOP = "STOP"
    STOP_LIMIT = "STOP_LIMIT"


class SignalType(Enum):
    """Signal types from strategy."""
    LONG = "LONG"
    SHORT = "SHORT"
    EXIT = "EXIT"
    HOLD = "HOLD"


# ============================================================================
# BASE EVENT CLASS
# ============================================================================

@dataclass
class Event:
    """
    Base class for all events in the trading system.
    
    All events have:
    - event_type: The category of the event
    - timestamp: When the event was created
    - event_id: Unique identifier for tracing
    """
    event_type: EventType
    timestamp: datetime = field(default_factory=datetime.now)
    event_id: str = field(default_factory=lambda: str(uuid.uuid4())[:8])
    
    def __post_init__(self):
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def to_dict(self) -> Dict[str, Any]:
        """Serialize event for logging/storage."""
        return {
            'event_type': self.event_type.name,
            'timestamp': self.timestamp.isoformat(),
            'event_id': self.event_id
        }


# ============================================================================
# CONCRETE EVENT CLASSES
# ============================================================================

@dataclass
class MarketDataEvent(Event):
    """
    Event containing new market data (tick or bar).
    
    Published by: DataHandler
    Consumed by: Strategy, RiskManager
    """
    symbol: str = ""
    open_price: float = 0.0
    high_price: float = 0.0
    low_price: float = 0.0
    close_price: float = 0.0
    volume: int = 0
    bid: float = 0.0
    ask: float = 0.0
    
    def __post_init__(self):
        self.event_type = EventType.MARKET_DATA
        super().__post_init__()
    
    @property
    def mid_price(self) -> float:
        """Calculate mid price from bid-ask."""
        return (self.bid + self.ask) / 2 if self.bid and self.ask else self.close_price
    
    @property
    def spread(self) -> float:
        """Calculate bid-ask spread."""
        return self.ask - self.bid if self.bid and self.ask else 0.0


@dataclass
class SignalEvent(Event):
    """
    Event representing a trading signal from strategy.
    
    Published by: Strategy
    Consumed by: Portfolio, RiskManager
    """
    symbol: str = ""
    signal_type: SignalType = SignalType.HOLD
    strength: float = 0.0  # Signal strength/confidence [0, 1]
    target_price: Optional[float] = None
    stop_loss: Optional[float] = None
    strategy_id: str = ""
    
    def __post_init__(self):
        self.event_type = EventType.SIGNAL
        super().__post_init__()


@dataclass
class OrderEvent(Event):
    """
    Event representing an order to be executed.
    
    Published by: Portfolio/RiskManager
    Consumed by: ExecutionHandler
    """
    symbol: str = ""
    order_type: OrderType = OrderType.MARKET
    side: OrderSide = OrderSide.BUY
    quantity: int = 0
    limit_price: Optional[float] = None
    stop_price: Optional[float] = None
    time_in_force: str = "DAY"  # DAY, GTC, IOC, FOK
    
    def __post_init__(self):
        self.event_type = EventType.ORDER
        super().__post_init__()
    
    def to_dict(self) -> Dict[str, Any]:
        base = super().to_dict()
        base.update({
            'symbol': self.symbol,
            'order_type': self.order_type.value,
            'side': self.side.value,
            'quantity': self.quantity,
            'limit_price': self.limit_price,
            'stop_price': self.stop_price
        })
        return base


@dataclass
class FillEvent(Event):
    """
    Event representing an order fill from exchange/broker.
    
    Published by: ExecutionHandler
    Consumed by: Portfolio, RiskManager
    """
    symbol: str = ""
    side: OrderSide = OrderSide.BUY
    quantity: int = 0
    fill_price: float = 0.0
    commission: float = 0.0
    exchange: str = ""
    order_id: str = ""
    
    def __post_init__(self):
        self.event_type = EventType.FILL
        super().__post_init__()
    
    @property
    def total_cost(self) -> float:
        """Calculate total cost including commission."""
        notional = self.quantity * self.fill_price
        if self.side == OrderSide.BUY:
            return notional + self.commission
        return -notional + self.commission


# Test event creation
print("=" * 60)
print("EVENT SYSTEM DEMONSTRATION")
print("=" * 60)

# Create sample events
market_event = MarketDataEvent(
    symbol="AAPL",
    close_price=150.00,
    bid=149.95,
    ask=150.05,
    volume=1000000
)
print(f"\n📊 Market Data Event: {market_event.symbol}")
print(f"   Mid Price: ${market_event.mid_price:.2f}")
print(f"   Spread: ${market_event.spread:.2f}")

signal_event = SignalEvent(
    symbol="AAPL",
    signal_type=SignalType.LONG,
    strength=0.85,
    strategy_id="momentum_v1"
)
print(f"\n📈 Signal Event: {signal_event.signal_type.value} on {signal_event.symbol}")
print(f"   Strength: {signal_event.strength:.2%}")

order_event = OrderEvent(
    symbol="AAPL",
    order_type=OrderType.LIMIT,
    side=OrderSide.BUY,
    quantity=100,
    limit_price=149.50
)
print(f"\n📝 Order Event: {order_event.side.value} {order_event.quantity} {order_event.symbol}")
print(f"   Type: {order_event.order_type.value} @ ${order_event.limit_price}")

fill_event = FillEvent(
    symbol="AAPL",
    side=OrderSide.BUY,
    quantity=100,
    fill_price=149.50,
    commission=1.00
)
print(f"\n✅ Fill Event: {fill_event.quantity} @ ${fill_event.fill_price}")
print(f"   Total Cost: ${fill_event.total_cost:.2f}")

EVENT SYSTEM DEMONSTRATION


TypeError: MarketDataEvent.__init__() missing 1 required positional argument: 'event_type'

### 2.2 Event Bus and Publish-Subscribe Pattern

The Event Bus is the central nervous system of the trading platform, routing events to appropriate handlers using the Observer/Pub-Sub pattern.

In [ ]:
# ============================================================================
# EVENT BUS - PUBLISH/SUBSCRIBE PATTERN
# ============================================================================

class EventBus:
    """
    Central event bus implementing publish-subscribe pattern.
    
    Features:
    - Multiple subscribers per event type
    - Priority-based event processing
    - Event history for replay/debugging
    - Thread-safe operations
    
    Usage:
        bus = EventBus()
        bus.subscribe(EventType.MARKET_DATA, my_handler)
        bus.publish(market_event)
    """
    
    def __init__(self, max_history: int = 10000):
        self._subscribers: Dict[EventType, List[Callable]] = {
            event_type: [] for event_type in EventType
        }
        self._event_queue: deque = deque()
        self._event_history: deque = deque(maxlen=max_history)
        self._lock = Lock()
        self._running = False
        self.logger = logging.getLogger("EventBus")
        
        # Metrics
        self._events_published = 0
        self._events_processed = 0
    
    def subscribe(
        self, 
        event_type: EventType, 
        handler: Callable[[Event], None],
        priority: int = 0
    ) -> None:
        """
        Subscribe a handler to an event type.
        
        Args:
            event_type: Type of event to subscribe to
            handler: Callable that takes an Event
            priority: Higher priority handlers called first
        """
        with self._lock:
            self._subscribers[event_type].append((priority, handler))
            # Sort by priority (descending)
            self._subscribers[event_type].sort(key=lambda x: -x[0])
            self.logger.info(
                f"Handler {handler.__name__} subscribed to {event_type.name}"
            )
    
    def unsubscribe(
        self, 
        event_type: EventType, 
        handler: Callable[[Event], None]
    ) -> None:
        """Remove a handler from event type."""
        with self._lock:
            self._subscribers[event_type] = [
                (p, h) for p, h in self._subscribers[event_type] 
                if h != handler
            ]
    
    def publish(self, event: Event) -> None:
        """
        Publish an event to the bus.
        
        Event is added to queue for processing and history for replay.
        """
        with self._lock:
            self._event_queue.append(event)
            self._event_history.append(event)
            self._events_published += 1
    
    def process_next(self) -> bool:
        """
        Process the next event in the queue.
        
        Returns:
            True if an event was processed, False if queue empty
        """
        if not self._event_queue:
            return False
        
        with self._lock:
            event = self._event_queue.popleft()
        
        # Dispatch to all subscribers for this event type
        handlers = self._subscribers.get(event.event_type, [])
        for priority, handler in handlers:
            try:
                handler(event)
            except Exception as e:
                self.logger.error(
                    f"Handler {handler.__name__} failed: {e}"
                )
        
        self._events_processed += 1
        return True
    
    def process_all(self) -> int:
        """Process all events in queue. Returns count processed."""
        count = 0
        while self.process_next():
            count += 1
        return count
    
    def get_stats(self) -> Dict[str, Any]:
        """Get event bus statistics."""
        return {
            'events_published': self._events_published,
            'events_processed': self._events_processed,
            'queue_depth': len(self._event_queue),
            'history_size': len(self._event_history),
            'subscribers': {
                et.name: len(handlers) 
                for et, handlers in self._subscribers.items()
            }
        }
    
    def replay_history(
        self, 
        start_time: Optional[datetime] = None,
        end_time: Optional[datetime] = None,
        event_types: Optional[List[EventType]] = None
    ) -> Iterator[Event]:
        """
        Replay events from history for debugging/testing.
        
        Args:
            start_time: Filter events after this time
            end_time: Filter events before this time
            event_types: Filter by event types
        """
        for event in self._event_history:
            if start_time and event.timestamp < start_time:
                continue
            if end_time and event.timestamp > end_time:
                continue
            if event_types and event.event_type not in event_types:
                continue
            yield event


# ============================================================================
# DEMONSTRATE EVENT BUS
# ============================================================================

print("=" * 60)
print("EVENT BUS DEMONSTRATION")
print("=" * 60)

# Create event bus
event_bus = EventBus()

# Create sample handlers
def market_data_logger(event: MarketDataEvent):
    print(f"   📊 [DataLogger] Received: {event.symbol} @ ${event.close_price}")

def signal_processor(event: SignalEvent):
    print(f"   📈 [SignalProc] Signal: {event.signal_type.value} {event.symbol}")

def order_validator(event: OrderEvent):
    print(f"   ✓ [Validator] Order validated: {event.side.value} {event.quantity}")

def fill_recorder(event: FillEvent):
    print(f"   💰 [Recorder] Fill: {event.quantity} @ ${event.fill_price}")

# Subscribe handlers
event_bus.subscribe(EventType.MARKET_DATA, market_data_logger)
event_bus.subscribe(EventType.SIGNAL, signal_processor)
event_bus.subscribe(EventType.ORDER, order_validator)
event_bus.subscribe(EventType.FILL, fill_recorder)

print("\n🔗 Handlers subscribed to event bus")

# Publish events
print("\n📤 Publishing events...")
event_bus.publish(market_event)
event_bus.publish(signal_event)
event_bus.publish(order_event)
event_bus.publish(fill_event)

# Process all events
print("\n⚙️ Processing events:")
processed = event_bus.process_all()

# Show statistics
print(f"\n📊 Event Bus Statistics:")
stats = event_bus.get_stats()
print(f"   Published: {stats['events_published']}")
print(f"   Processed: {stats['events_processed']}")
print(f"   Queue Depth: {stats['queue_depth']}")

## 3. Component-Based System Design

Each component in the trading system follows the Interface Segregation Principle - defined through abstract base classes with clear contracts.

### 3.1 Core Component Interfaces

```
Component Hierarchy:
───────────────────────────────────────────────
        ┌──────────────────┐
        │  TradingSystem   │
        │    (Facade)      │
        └────────┬─────────┘
                 │
    ┌────────────┼────────────┐
    │            │            │
    ▼            ▼            ▼
┌───────┐   ┌────────┐   ┌──────────┐
│ Data  │   │Strategy│   │Execution │
│Handler│   │ Engine │   │ Handler  │
└───────┘   └────────┘   └──────────┘
    │            │            │
    └────────────┼────────────┘
                 │
        ┌────────┴────────┐
        │                 │
        ▼                 ▼
   ┌─────────┐      ┌──────────┐
   │Portfolio│      │   Risk   │
   │ Manager │      │ Manager  │
   └─────────┘      └──────────┘
```

In [ ]:
# ============================================================================
# ABSTRACT BASE COMPONENTS - INTERFACE DEFINITIONS
# ============================================================================

class DataHandler(ABC):
    """
    Abstract base class for data handling components.
    
    Responsibilities:
    - Fetch market data from various sources
    - Normalize data format
    - Publish MarketDataEvents
    
    Concrete implementations: HistoricalDataHandler, LiveDataHandler
    """
    
    @abstractmethod
    def get_latest_bar(self, symbol: str) -> Optional[Dict[str, Any]]:
        """Get the most recent bar for a symbol."""
        pass
    
    @abstractmethod
    def get_latest_bars(self, symbol: str, n: int = 1) -> List[Dict[str, Any]]:
        """Get the N most recent bars for a symbol."""
        pass
    
    @abstractmethod
    def update_bars(self) -> None:
        """Push the latest bar to the system."""
        pass
    
    @property
    @abstractmethod
    def continue_backtest(self) -> bool:
        """Check if more data is available."""
        pass


class Strategy(ABC):
    """
    Abstract base class for trading strategies.
    
    Responsibilities:
    - Analyze market data
    - Generate trading signals
    - Manage strategy state
    
    Follows Strategy Pattern for interchangeable algorithms.
    """
    
    @abstractmethod
    def calculate_signals(self, event: MarketDataEvent) -> Optional[SignalEvent]:
        """
        Process market data and generate signals.
        
        Args:
            event: MarketDataEvent with latest data
            
        Returns:
            SignalEvent if signal generated, None otherwise
        """
        pass
    
    @property
    @abstractmethod
    def strategy_id(self) -> str:
        """Unique identifier for the strategy."""
        pass
    
    @abstractmethod
    def reset(self) -> None:
        """Reset strategy state."""
        pass


class Portfolio(ABC):
    """
    Abstract base class for portfolio management.
    
    Responsibilities:
    - Track positions and holdings
    - Convert signals to orders
    - Calculate P&L and metrics
    """
    
    @abstractmethod
    def update_signal(self, event: SignalEvent) -> Optional[OrderEvent]:
        """
        Convert signal to order based on portfolio logic.
        
        Args:
            event: SignalEvent from strategy
            
        Returns:
            OrderEvent if order should be placed
        """
        pass
    
    @abstractmethod
    def update_fill(self, event: FillEvent) -> None:
        """Update portfolio based on fill."""
        pass
    
    @property
    @abstractmethod
    def current_positions(self) -> Dict[str, int]:
        """Get current positions for all symbols."""
        pass
    
    @property
    @abstractmethod
    def equity_curve(self) -> pd.Series:
        """Get equity curve time series."""
        pass


class ExecutionHandler(ABC):
    """
    Abstract base class for order execution.
    
    Responsibilities:
    - Send orders to exchange/broker
    - Handle order lifecycle
    - Generate fill events
    """
    
    @abstractmethod
    def execute_order(self, event: OrderEvent) -> Optional[FillEvent]:
        """
        Execute an order and return fill.
        
        Args:
            event: OrderEvent to execute
            
        Returns:
            FillEvent with execution details
        """
        pass
    
    @abstractmethod
    def cancel_order(self, order_id: str) -> bool:
        """Cancel an open order."""
        pass


class RiskManager(ABC):
    """
    Abstract base class for risk management.
    
    Responsibilities:
    - Pre-trade risk checks
    - Position limit enforcement
    - Exposure monitoring
    """
    
    @abstractmethod
    def check_order(self, event: OrderEvent) -> bool:
        """
        Validate order against risk limits.
        
        Returns:
            True if order passes risk checks
        """
        pass
    
    @abstractmethod
    def get_risk_metrics(self) -> Dict[str, float]:
        """Get current risk metrics."""
        pass


print("=" * 60)
print("ABSTRACT COMPONENT INTERFACES DEFINED")
print("=" * 60)
print("\n✅ DataHandler - Market data management")
print("✅ Strategy - Signal generation")
print("✅ Portfolio - Position and P&L tracking")
print("✅ ExecutionHandler - Order execution")
print("✅ RiskManager - Risk controls")

### 3.2 Concrete Component Implementations

In [ ]:
# ============================================================================
# CONCRETE IMPLEMENTATIONS
# ============================================================================

class HistoricalDataHandler(DataHandler):
    """
    Data handler for backtesting with historical data.
    
    Simulates streaming data by iterating through historical bars.
    """
    
    def __init__(self, data: pd.DataFrame, event_bus: EventBus):
        """
        Args:
            data: DataFrame with OHLCV data, indexed by datetime
            event_bus: EventBus to publish market data events
        """
        self.data = data
        self.event_bus = event_bus
        self.symbols = data['symbol'].unique().tolist() if 'symbol' in data.columns else ['UNKNOWN']
        self._current_idx = 0
        self._latest_bars: Dict[str, List[Dict]] = {s: [] for s in self.symbols}
        self.logger = logging.getLogger("HistoricalDataHandler")
    
    def get_latest_bar(self, symbol: str) -> Optional[Dict[str, Any]]:
        """Get the most recent bar for symbol."""
        bars = self._latest_bars.get(symbol, [])
        return bars[-1] if bars else None
    
    def get_latest_bars(self, symbol: str, n: int = 1) -> List[Dict[str, Any]]:
        """Get N most recent bars for symbol."""
        bars = self._latest_bars.get(symbol, [])
        return bars[-n:] if bars else []
    
    def update_bars(self) -> None:
        """Push the next bar from historical data."""
        if self._current_idx >= len(self.data):
            return
        
        row = self.data.iloc[self._current_idx]
        symbol = row.get('symbol', self.symbols[0])
        
        bar = {
            'timestamp': row.name if isinstance(row.name, datetime) else datetime.now(),
            'open': row.get('open', row.get('Open', 0)),
            'high': row.get('high', row.get('High', 0)),
            'low': row.get('low', row.get('Low', 0)),
            'close': row.get('close', row.get('Close', 0)),
            'volume': row.get('volume', row.get('Volume', 0)),
            'symbol': symbol
        }
        
        self._latest_bars[symbol].append(bar)
        
        # Publish market data event
        event = MarketDataEvent(
            symbol=symbol,
            open_price=bar['open'],
            high_price=bar['high'],
            low_price=bar['low'],
            close_price=bar['close'],
            volume=bar['volume'],
            timestamp=bar['timestamp']
        )
        self.event_bus.publish(event)
        
        self._current_idx += 1
    
    @property
    def continue_backtest(self) -> bool:
        """Check if more data available."""
        return self._current_idx < len(self.data)


class MomentumStrategy(Strategy):
    """
    Simple momentum strategy for demonstration.
    
    Generates LONG signal when short MA > long MA.
    Generates SHORT signal when short MA < long MA.
    """
    
    def __init__(
        self, 
        short_window: int = 10, 
        long_window: int = 30,
        strategy_name: str = "momentum_v1"
    ):
        self.short_window = short_window
        self.long_window = long_window
        self._strategy_id = strategy_name
        self._prices: Dict[str, List[float]] = {}
        self._current_signal: Dict[str, SignalType] = {}
        self.logger = logging.getLogger("MomentumStrategy")
    
    @property
    def strategy_id(self) -> str:
        return self._strategy_id
    
    def calculate_signals(self, event: MarketDataEvent) -> Optional[SignalEvent]:
        """Calculate momentum signals."""
        symbol = event.symbol
        price = event.close_price
        
        # Initialize price history
        if symbol not in self._prices:
            self._prices[symbol] = []
            self._current_signal[symbol] = SignalType.HOLD
        
        self._prices[symbol].append(price)
        
        # Need enough data for long MA
        if len(self._prices[symbol]) < self.long_window:
            return None
        
        # Calculate moving averages
        prices = self._prices[symbol]
        short_ma = np.mean(prices[-self.short_window:])
        long_ma = np.mean(prices[-self.long_window:])
        
        # Determine signal
        prev_signal = self._current_signal[symbol]
        
        if short_ma > long_ma and prev_signal != SignalType.LONG:
            self._current_signal[symbol] = SignalType.LONG
            return SignalEvent(
                symbol=symbol,
                signal_type=SignalType.LONG,
                strength=min(1.0, (short_ma - long_ma) / long_ma * 100),
                strategy_id=self._strategy_id
            )
        elif short_ma < long_ma and prev_signal != SignalType.SHORT:
            self._current_signal[symbol] = SignalType.SHORT
            return SignalEvent(
                symbol=symbol,
                signal_type=SignalType.SHORT,
                strength=min(1.0, (long_ma - short_ma) / long_ma * 100),
                strategy_id=self._strategy_id
            )
        
        return None
    
    def reset(self) -> None:
        """Reset strategy state."""
        self._prices.clear()
        self._current_signal.clear()


class SimplePortfolio(Portfolio):
    """
    Simple portfolio that converts signals to fixed-size orders.
    """
    
    def __init__(
        self, 
        initial_capital: float = 100000.0,
        position_size: int = 100
    ):
        self.initial_capital = initial_capital
        self.position_size = position_size
        self.cash = initial_capital
        self._positions: Dict[str, int] = {}
        self._holdings: Dict[str, float] = {}
        self._equity_history: List[Dict] = []
        self.logger = logging.getLogger("SimplePortfolio")
    
    @property
    def current_positions(self) -> Dict[str, int]:
        return self._positions.copy()
    
    @property
    def equity_curve(self) -> pd.Series:
        if not self._equity_history:
            return pd.Series([self.initial_capital])
        return pd.Series([e['equity'] for e in self._equity_history])
    
    def update_signal(self, event: SignalEvent) -> Optional[OrderEvent]:
        """Convert signal to order."""
        symbol = event.symbol
        current_pos = self._positions.get(symbol, 0)
        
        if event.signal_type == SignalType.LONG and current_pos <= 0:
            # Go long or cover short
            quantity = self.position_size + abs(current_pos)
            return OrderEvent(
                symbol=symbol,
                order_type=OrderType.MARKET,
                side=OrderSide.BUY,
                quantity=quantity
            )
        elif event.signal_type == SignalType.SHORT and current_pos >= 0:
            # Go short or sell long
            quantity = self.position_size + abs(current_pos)
            return OrderEvent(
                symbol=symbol,
                order_type=OrderType.MARKET,
                side=OrderSide.SELL,
                quantity=quantity
            )
        elif event.signal_type == SignalType.EXIT and current_pos != 0:
            # Exit position
            return OrderEvent(
                symbol=symbol,
                order_type=OrderType.MARKET,
                side=OrderSide.SELL if current_pos > 0 else OrderSide.BUY,
                quantity=abs(current_pos)
            )
        
        return None
    
    def update_fill(self, event: FillEvent) -> None:
        """Update portfolio with fill."""
        symbol = event.symbol
        
        if symbol not in self._positions:
            self._positions[symbol] = 0
            self._holdings[symbol] = 0.0
        
        # Update position
        if event.side == OrderSide.BUY:
            self._positions[symbol] += event.quantity
            self.cash -= event.total_cost
        else:
            self._positions[symbol] -= event.quantity
            self.cash += event.quantity * event.fill_price - event.commission
        
        self._holdings[symbol] = event.fill_price
        
        # Record equity
        total_equity = self.cash + sum(
            pos * self._holdings.get(sym, 0) 
            for sym, pos in self._positions.items()
        )
        self._equity_history.append({
            'timestamp': event.timestamp,
            'equity': total_equity,
            'cash': self.cash
        })


class SimulatedExecutionHandler(ExecutionHandler):
    """
    Simulated execution handler for backtesting.
    
    Simulates fills with configurable slippage and commission.
    """
    
    def __init__(
        self, 
        slippage_pct: float = 0.001,
        commission_per_share: float = 0.01
    ):
        self.slippage_pct = slippage_pct
        self.commission_per_share = commission_per_share
        self._latest_prices: Dict[str, float] = {}
        self._open_orders: Dict[str, OrderEvent] = {}
        self.logger = logging.getLogger("SimulatedExecutionHandler")
    
    def update_price(self, symbol: str, price: float) -> None:
        """Update latest price for symbol."""
        self._latest_prices[symbol] = price
    
    def execute_order(self, event: OrderEvent) -> Optional[FillEvent]:
        """Simulate order execution."""
        symbol = event.symbol
        
        if symbol not in self._latest_prices:
            self.logger.warning(f"No price available for {symbol}")
            return None
        
        base_price = self._latest_prices[symbol]
        
        # Apply slippage
        if event.side == OrderSide.BUY:
            fill_price = base_price * (1 + self.slippage_pct)
        else:
            fill_price = base_price * (1 - self.slippage_pct)
        
        # Calculate commission
        commission = event.quantity * self.commission_per_share
        
        return FillEvent(
            symbol=symbol,
            side=event.side,
            quantity=event.quantity,
            fill_price=fill_price,
            commission=commission,
            order_id=event.event_id
        )
    
    def cancel_order(self, order_id: str) -> bool:
        """Cancel open order."""
        if order_id in self._open_orders:
            del self._open_orders[order_id]
            return True
        return False


print("=" * 60)
print("CONCRETE COMPONENT IMPLEMENTATIONS")
print("=" * 60)
print("\n✅ HistoricalDataHandler - Backtest data streaming")
print("✅ MomentumStrategy - MA crossover signals")
print("✅ SimplePortfolio - Position and P&L tracking")
print("✅ SimulatedExecutionHandler - Simulated fills")

## 4. Data Flow Pipeline Architecture

Efficient data pipelines are crucial for trading systems. We use generators for memory-efficient streaming and clean separation of concerns.

```
Data Pipeline Stages:
───────────────────────────────────────────────────────────────
Raw Data → Cleaning → Feature Engineering → Signal Generation
    │          │              │                    │
    ▼          ▼              ▼                    ▼
 DataFetcher  DataCleaner  FeatureEngine  SignalGenerator
    │          │              │                    │
    └──────────┴──────────────┴────────────────────┘
                       │
              Generator Pattern
           (Memory-Efficient Streaming)
```

In [ ]:
# ============================================================================
# DATA PIPELINE COMPONENTS
# ============================================================================

@dataclass
class Bar:
    """Immutable market data bar."""
    timestamp: datetime
    symbol: str
    open: float
    high: float
    low: float
    close: float
    volume: int
    
    @property
    def typical_price(self) -> float:
        """Calculate typical price."""
        return (self.high + self.low + self.close) / 3
    
    @property
    def range(self) -> float:
        """Calculate bar range."""
        return self.high - self.low


class DataFetcher:
    """
    First stage: Fetch raw data from source.
    
    Supports multiple data sources through adapter pattern.
    """
    
    def __init__(self, source: str = "csv"):
        self.source = source
        self.logger = logging.getLogger("DataFetcher")
    
    def fetch_bars(self, data: pd.DataFrame) -> Iterator[Bar]:
        """
        Generator that yields bars from data source.
        
        Memory efficient - only one bar in memory at a time.
        """
        for idx, row in data.iterrows():
            yield Bar(
                timestamp=idx if isinstance(idx, datetime) else datetime.now(),
                symbol=row.get('symbol', 'UNKNOWN'),
                open=float(row.get('open', row.get('Open', 0))),
                high=float(row.get('high', row.get('High', 0))),
                low=float(row.get('low', row.get('Low', 0))),
                close=float(row.get('close', row.get('Close', 0))),
                volume=int(row.get('volume', row.get('Volume', 0)))
            )


class DataCleaner:
    """
    Second stage: Clean and validate data.
    
    Handles missing values, outliers, and data quality issues.
    """
    
    def __init__(self, max_price_change: float = 0.5):
        self.max_price_change = max_price_change
        self._prev_bar: Optional[Bar] = None
        self.logger = logging.getLogger("DataCleaner")
    
    def clean(self, bars: Iterator[Bar]) -> Iterator[Bar]:
        """
        Generator that filters and cleans bars.
        
        Removes:
        - Bars with invalid prices (<=0)
        - Bars with extreme price changes (potential bad ticks)
        """
        for bar in bars:
            # Skip invalid bars
            if bar.close <= 0 or bar.open <= 0:
                self.logger.warning(f"Invalid price in bar: {bar}")
                continue
            
            # Check for extreme price changes
            if self._prev_bar is not None:
                price_change = abs(bar.close - self._prev_bar.close) / self._prev_bar.close
                if price_change > self.max_price_change:
                    self.logger.warning(
                        f"Extreme price change: {price_change:.2%} for {bar.symbol}"
                    )
                    continue
            
            self._prev_bar = bar
            yield bar


class FeatureEngine:
    """
    Third stage: Calculate trading features.
    
    Adds technical indicators and derived features to bars.
    """
    
    def __init__(self, window_sizes: List[int] = [5, 10, 20]):
        self.window_sizes = window_sizes
        self._price_history: List[float] = []
        self._volume_history: List[int] = []
        self.logger = logging.getLogger("FeatureEngine")
    
    def add_features(self, bars: Iterator[Bar]) -> Iterator[Dict[str, Any]]:
        """
        Generator that enriches bars with features.
        
        Features added:
        - Moving averages (multiple windows)
        - Volatility estimates
        - Volume metrics
        - Momentum indicators
        """
        for bar in bars:
            self._price_history.append(bar.close)
            self._volume_history.append(bar.volume)
            
            features = {
                'bar': bar,
                'timestamp': bar.timestamp,
                'symbol': bar.symbol,
                'close': bar.close,
                'volume': bar.volume
            }
            
            # Calculate moving averages
            for window in self.window_sizes:
                if len(self._price_history) >= window:
                    features[f'sma_{window}'] = np.mean(self._price_history[-window:])
                    features[f'vol_{window}'] = np.std(self._price_history[-window:])
                    features[f'avg_volume_{window}'] = np.mean(self._volume_history[-window:])
            
            # Returns
            if len(self._price_history) >= 2:
                features['return_1'] = (bar.close / self._price_history[-2]) - 1
            
            if len(self._price_history) >= 5:
                features['return_5'] = (bar.close / self._price_history[-5]) - 1
            
            # Momentum
            if len(self._price_history) >= 10:
                features['momentum_10'] = bar.close - self._price_history[-10]
            
            yield features


class SignalGenerator:
    """
    Fourth stage: Generate trading signals from features.
    
    Applies strategy logic to enriched data.
    """
    
    def __init__(self, strategy: Strategy):
        self.strategy = strategy
        self.logger = logging.getLogger("SignalGenerator")
    
    def generate(self, features: Iterator[Dict]) -> Iterator[tuple]:
        """
        Generator that produces signals from features.
        
        Yields (features, signal) tuples.
        """
        for feat in features:
            bar = feat['bar']
            
            # Create market data event for strategy
            event = MarketDataEvent(
                symbol=bar.symbol,
                open_price=bar.open,
                high_price=bar.high,
                low_price=bar.low,
                close_price=bar.close,
                volume=bar.volume,
                timestamp=bar.timestamp
            )
            
            # Get signal from strategy
            signal = self.strategy.calculate_signals(event)
            
            yield feat, signal


# ============================================================================
# PIPELINE ORCHESTRATOR
# ============================================================================

class DataPipeline:
    """
    Orchestrates the complete data pipeline.
    
    Chains generators together for efficient streaming processing.
    """
    
    def __init__(
        self,
        fetcher: DataFetcher,
        cleaner: DataCleaner,
        feature_engine: FeatureEngine,
        signal_generator: SignalGenerator
    ):
        self.fetcher = fetcher
        self.cleaner = cleaner
        self.feature_engine = feature_engine
        self.signal_generator = signal_generator
        self.logger = logging.getLogger("DataPipeline")
    
    def process(self, data: pd.DataFrame) -> Iterator[tuple]:
        """
        Process data through the complete pipeline.
        
        Uses generator chaining for memory efficiency:
        fetch -> clean -> features -> signals
        """
        # Chain generators
        raw_bars = self.fetcher.fetch_bars(data)
        clean_bars = self.cleaner.clean(raw_bars)
        enriched = self.feature_engine.add_features(clean_bars)
        signals = self.signal_generator.generate(enriched)
        
        # Yield results
        for result in signals:
            yield result


# ============================================================================
# DEMONSTRATE PIPELINE
# ============================================================================

print("=" * 60)
print("DATA PIPELINE DEMONSTRATION")
print("=" * 60)

# Create sample data
np.random.seed(42)
dates = pd.date_range('2024-01-01', periods=50, freq='D')
prices = 100 * np.cumprod(1 + np.random.randn(50) * 0.02)

sample_data = pd.DataFrame({
    'symbol': 'AAPL',
    'open': prices * (1 + np.random.randn(50) * 0.01),
    'high': prices * (1 + np.abs(np.random.randn(50)) * 0.01),
    'low': prices * (1 - np.abs(np.random.randn(50)) * 0.01),
    'close': prices,
    'volume': np.random.randint(1000000, 5000000, 50)
}, index=dates)

# Create pipeline components
fetcher = DataFetcher()
cleaner = DataCleaner()
feature_engine = FeatureEngine(window_sizes=[5, 10, 20])
strategy = MomentumStrategy(short_window=5, long_window=20)
signal_gen = SignalGenerator(strategy)

# Create pipeline
pipeline = DataPipeline(fetcher, cleaner, feature_engine, signal_gen)

# Process data
print("\n🔄 Processing data through pipeline...")
signals_found = 0
for i, (features, signal) in enumerate(pipeline.process(sample_data)):
    if signal is not None:
        signals_found += 1
        print(f"   📊 Day {i+1}: {signal.signal_type.value} signal "
              f"(strength: {signal.strength:.2%})")

print(f"\n✅ Pipeline processed 50 bars, found {signals_found} signals")
print("💡 Memory efficient: Only one bar in memory at a time")

## 5. Order Management System (OMS) Structure

The OMS is the central hub for order lifecycle management. It tracks orders from creation to fill/cancellation and maintains the order book.

```
Order Lifecycle State Machine:
───────────────────────────────────────────────────────────────
                    ┌─────────────┐
                    │   CREATED   │
                    └──────┬──────┘
                           │ submit()
                           ▼
                    ┌─────────────┐
            ┌───────│   PENDING   │───────┐
            │       └──────┬──────┘       │
            │ cancel()     │ accept()     │ reject()
            │              ▼              │
            │       ┌─────────────┐       │
            │       │  SUBMITTED  │───────┤
            │       └──────┬──────┘       │
            │              │              │
            ▼       ┌──────┴──────┐       ▼
     ┌──────────┐   │             │   ┌──────────┐
     │CANCELLED │   ▼             ▼   │ REJECTED │
     └──────────┘ ┌────┐       ┌──────┴──────┐
                  │PART│       │   FILLED    │
                  │FILL│       └─────────────┘
                  └────┘
```

In [ ]:
# ============================================================================
# ORDER MANAGEMENT SYSTEM
# ============================================================================

class OrderStatus(Enum):
    """Order lifecycle states."""
    CREATED = "CREATED"
    PENDING = "PENDING"
    SUBMITTED = "SUBMITTED"
    PARTIALLY_FILLED = "PARTIALLY_FILLED"
    FILLED = "FILLED"
    CANCELLED = "CANCELLED"
    REJECTED = "REJECTED"
    EXPIRED = "EXPIRED"


@dataclass
class Order:
    """
    Complete order representation with full lifecycle tracking.
    """
    order_id: str
    symbol: str
    side: OrderSide
    order_type: OrderType
    quantity: int
    limit_price: Optional[float] = None
    stop_price: Optional[float] = None
    time_in_force: str = "DAY"
    
    # State tracking
    status: OrderStatus = OrderStatus.CREATED
    filled_quantity: int = 0
    average_fill_price: float = 0.0
    
    # Timestamps
    created_at: datetime = field(default_factory=datetime.now)
    submitted_at: Optional[datetime] = None
    filled_at: Optional[datetime] = None
    
    # Audit trail
    status_history: List[tuple] = field(default_factory=list)
    
    def __post_init__(self):
        self._record_status_change()
    
    def _record_status_change(self) -> None:
        """Record status transition for audit."""
        self.status_history.append((datetime.now(), self.status))
    
    @property
    def remaining_quantity(self) -> int:
        """Quantity yet to be filled."""
        return self.quantity - self.filled_quantity
    
    @property
    def is_active(self) -> bool:
        """Check if order is still active."""
        return self.status in [
            OrderStatus.PENDING, 
            OrderStatus.SUBMITTED,
            OrderStatus.PARTIALLY_FILLED
        ]
    
    @property
    def notional_value(self) -> float:
        """Calculate notional value of order."""
        price = self.limit_price or self.average_fill_price or 0
        return self.quantity * price
    
    def submit(self) -> bool:
        """Transition to PENDING state."""
        if self.status == OrderStatus.CREATED:
            self.status = OrderStatus.PENDING
            self.submitted_at = datetime.now()
            self._record_status_change()
            return True
        return False
    
    def accept(self) -> bool:
        """Exchange accepted order."""
        if self.status == OrderStatus.PENDING:
            self.status = OrderStatus.SUBMITTED
            self._record_status_change()
            return True
        return False
    
    def fill(self, quantity: int, price: float) -> bool:
        """
        Record a fill against this order.
        
        Handles partial fills automatically.
        """
        if not self.is_active:
            return False
        
        # Update average fill price
        total_filled_value = (
            self.average_fill_price * self.filled_quantity + 
            price * quantity
        )
        self.filled_quantity += quantity
        self.average_fill_price = total_filled_value / self.filled_quantity
        
        # Update status
        if self.filled_quantity >= self.quantity:
            self.status = OrderStatus.FILLED
            self.filled_at = datetime.now()
        else:
            self.status = OrderStatus.PARTIALLY_FILLED
        
        self._record_status_change()
        return True
    
    def cancel(self) -> bool:
        """Cancel the order."""
        if self.is_active:
            self.status = OrderStatus.CANCELLED
            self._record_status_change()
            return True
        return False
    
    def reject(self, reason: str = "") -> bool:
        """Reject the order."""
        if self.status == OrderStatus.PENDING:
            self.status = OrderStatus.REJECTED
            self._record_status_change()
            return True
        return False


class OrderBook:
    """
    Order book management for a single symbol.
    
    Maintains buy and sell orders with price-time priority.
    """
    
    def __init__(self, symbol: str):
        self.symbol = symbol
        self._bids: List[Order] = []  # Buy orders (descending price)
        self._asks: List[Order] = []  # Sell orders (ascending price)
        self.logger = logging.getLogger(f"OrderBook_{symbol}")
    
    def add_order(self, order: Order) -> None:
        """Add order to book with price-time priority."""
        if order.symbol != self.symbol:
            raise ValueError(f"Order symbol {order.symbol} != book {self.symbol}")
        
        if order.side == OrderSide.BUY:
            self._bids.append(order)
            # Sort by price descending, then time ascending
            self._bids.sort(
                key=lambda o: (-o.limit_price if o.limit_price else float('inf'), 
                              o.created_at)
            )
        else:
            self._asks.append(order)
            # Sort by price ascending, then time ascending
            self._asks.sort(
                key=lambda o: (o.limit_price if o.limit_price else float('inf'), 
                              o.created_at)
            )
    
    def remove_order(self, order_id: str) -> Optional[Order]:
        """Remove order from book."""
        for order_list in [self._bids, self._asks]:
            for i, order in enumerate(order_list):
                if order.order_id == order_id:
                    return order_list.pop(i)
        return None
    
    @property
    def best_bid(self) -> Optional[Order]:
        """Get best (highest) bid."""
        active_bids = [o for o in self._bids if o.is_active]
        return active_bids[0] if active_bids else None
    
    @property
    def best_ask(self) -> Optional[Order]:
        """Get best (lowest) ask."""
        active_asks = [o for o in self._asks if o.is_active]
        return active_asks[0] if active_asks else None
    
    @property
    def spread(self) -> Optional[float]:
        """Calculate bid-ask spread."""
        if self.best_bid and self.best_ask:
            return self.best_ask.limit_price - self.best_bid.limit_price
        return None
    
    @property
    def mid_price(self) -> Optional[float]:
        """Calculate mid price."""
        if self.best_bid and self.best_ask:
            return (self.best_bid.limit_price + self.best_ask.limit_price) / 2
        return None


class OrderManagementSystem:
    """
    Central OMS for managing all orders across symbols.
    
    Features:
    - Order creation and validation
    - Order book management per symbol
    - Position tracking
    - Order history and audit
    """
    
    def __init__(self, event_bus: Optional[EventBus] = None):
        self.event_bus = event_bus
        self._orders: Dict[str, Order] = {}  # order_id -> Order
        self._order_books: Dict[str, OrderBook] = {}  # symbol -> OrderBook
        self._positions: Dict[str, int] = {}  # symbol -> position
        self._order_counter = 0
        self.logger = logging.getLogger("OMS")
    
    def _generate_order_id(self) -> str:
        """Generate unique order ID."""
        self._order_counter += 1
        return f"ORD-{self._order_counter:06d}"
    
    def _get_order_book(self, symbol: str) -> OrderBook:
        """Get or create order book for symbol."""
        if symbol not in self._order_books:
            self._order_books[symbol] = OrderBook(symbol)
        return self._order_books[symbol]
    
    def create_order(
        self,
        symbol: str,
        side: OrderSide,
        quantity: int,
        order_type: OrderType = OrderType.MARKET,
        limit_price: Optional[float] = None,
        stop_price: Optional[float] = None
    ) -> Order:
        """
        Create a new order.
        
        Returns:
            Order object (not yet submitted)
        """
        order = Order(
            order_id=self._generate_order_id(),
            symbol=symbol,
            side=side,
            order_type=order_type,
            quantity=quantity,
            limit_price=limit_price,
            stop_price=stop_price
        )
        
        self._orders[order.order_id] = order
        self.logger.info(f"Created order: {order.order_id}")
        
        return order
    
    def submit_order(self, order_id: str) -> bool:
        """Submit order for execution."""
        order = self._orders.get(order_id)
        if not order:
            return False
        
        if order.submit():
            # Add to order book for limit orders
            if order.order_type == OrderType.LIMIT:
                book = self._get_order_book(order.symbol)
                book.add_order(order)
            
            self.logger.info(f"Submitted order: {order_id}")
            return True
        return False
    
    def fill_order(
        self, 
        order_id: str, 
        quantity: int, 
        price: float
    ) -> Optional[FillEvent]:
        """
        Record a fill against an order.
        
        Returns:
            FillEvent if fill successful
        """
        order = self._orders.get(order_id)
        if not order:
            return None
        
        if order.fill(quantity, price):
            # Update position
            if order.symbol not in self._positions:
                self._positions[order.symbol] = 0
            
            if order.side == OrderSide.BUY:
                self._positions[order.symbol] += quantity
            else:
                self._positions[order.symbol] -= quantity
            
            # Create fill event
            fill = FillEvent(
                symbol=order.symbol,
                side=order.side,
                quantity=quantity,
                fill_price=price,
                commission=0.01 * quantity,  # $0.01/share
                order_id=order_id
            )
            
            # Publish if event bus available
            if self.event_bus:
                self.event_bus.publish(fill)
            
            self.logger.info(
                f"Filled {quantity} @ ${price:.2f} for {order_id}"
            )
            
            return fill
        return None
    
    def cancel_order(self, order_id: str) -> bool:
        """Cancel an open order."""
        order = self._orders.get(order_id)
        if order and order.cancel():
            # Remove from order book
            book = self._get_order_book(order.symbol)
            book.remove_order(order_id)
            
            self.logger.info(f"Cancelled order: {order_id}")
            return True
        return False
    
    def get_position(self, symbol: str) -> int:
        """Get current position for symbol."""
        return self._positions.get(symbol, 0)
    
    def get_open_orders(self, symbol: Optional[str] = None) -> List[Order]:
        """Get all open orders, optionally filtered by symbol."""
        return [
            o for o in self._orders.values()
            if o.is_active and (symbol is None or o.symbol == symbol)
        ]
    
    def get_order_stats(self) -> Dict[str, Any]:
        """Get OMS statistics."""
        orders = list(self._orders.values())
        return {
            'total_orders': len(orders),
            'active_orders': len([o for o in orders if o.is_active]),
            'filled_orders': len([o for o in orders if o.status == OrderStatus.FILLED]),
            'cancelled_orders': len([o for o in orders if o.status == OrderStatus.CANCELLED]),
            'positions': self._positions.copy()
        }


# ============================================================================
# DEMONSTRATE OMS
# ============================================================================

print("=" * 60)
print("ORDER MANAGEMENT SYSTEM DEMONSTRATION")
print("=" * 60)

# Create OMS
oms = OrderManagementSystem()

# Create and submit orders
print("\n📝 Creating orders...")
order1 = oms.create_order("AAPL", OrderSide.BUY, 100, OrderType.LIMIT, limit_price=150.00)
order2 = oms.create_order("AAPL", OrderSide.BUY, 50, OrderType.LIMIT, limit_price=149.50)
order3 = oms.create_order("AAPL", OrderSide.SELL, 75, OrderType.LIMIT, limit_price=151.00)

print(f"   Created: {order1.order_id} - BUY 100 AAPL @ $150.00")
print(f"   Created: {order2.order_id} - BUY 50 AAPL @ $149.50")
print(f"   Created: {order3.order_id} - SELL 75 AAPL @ $151.00")

# Submit orders
print("\n📤 Submitting orders...")
for order in [order1, order2, order3]:
    oms.submit_order(order.order_id)
    order.accept()  # Simulate exchange acceptance

# Check order book
print("\n📖 Order Book State:")
book = oms._get_order_book("AAPL")
print(f"   Best Bid: ${book.best_bid.limit_price:.2f}" if book.best_bid else "   No bids")
print(f"   Best Ask: ${book.best_ask.limit_price:.2f}" if book.best_ask else "   No asks")
print(f"   Spread: ${book.spread:.2f}" if book.spread else "   No spread")

# Simulate fills
print("\n✅ Executing fills...")
fill1 = oms.fill_order(order1.order_id, 100, 150.05)
print(f"   Filled {order1.order_id}: 100 @ $150.05")

fill2 = oms.fill_order(order2.order_id, 25, 149.55)  # Partial fill
print(f"   Partial fill {order2.order_id}: 25 @ $149.55")
print(f"   Order status: {order2.status.value}, Remaining: {order2.remaining_quantity}")

# Cancel remaining
oms.cancel_order(order2.order_id)
print(f"   Cancelled remaining on {order2.order_id}")

# Show stats
print("\n📊 OMS Statistics:")
stats = oms.get_order_stats()
print(f"   Total Orders: {stats['total_orders']}")
print(f"   Filled: {stats['filled_orders']}")
print(f"   Cancelled: {stats['cancelled_orders']}")
print(f"   Active: {stats['active_orders']}")
print(f"   Position AAPL: {stats['positions'].get('AAPL', 0)}")

## 6. Risk Management Module Integration

Risk management is a critical gatekeeper that validates all orders before execution. It implements pre-trade and post-trade risk controls.

```
Risk Management Layers:
───────────────────────────────────────────────────────────────
Layer 1: PRE-TRADE CHECKS
├── Position Limits     - Max position per symbol
├── Notional Limits    - Max order value
├── Concentration      - Max % of portfolio in single name
└── Order Rate         - Max orders per second

Layer 2: REAL-TIME MONITORING
├── P&L Limits         - Daily loss limits
├── Drawdown           - Max drawdown threshold
├── VaR/CVaR           - Value at Risk limits
└── Margin             - Margin utilization

Layer 3: POST-TRADE
├── Position Reconciliation
├── Fill Analysis
└── Exposure Reports
```

In [ ]:
# ============================================================================
# RISK MANAGEMENT SYSTEM
# ============================================================================

@dataclass
class RiskLimits:
    """Configuration for risk limits."""
    max_position_size: int = 1000          # Max shares per symbol
    max_notional_order: float = 100000.0   # Max order value
    max_concentration: float = 0.20        # Max % of portfolio per symbol
    max_daily_loss: float = 10000.0        # Daily loss limit
    max_drawdown: float = 0.10             # Max drawdown (10%)
    max_orders_per_minute: int = 60        # Rate limit


@dataclass 
class RiskCheckResult:
    """Result of a risk check."""
    passed: bool
    check_name: str
    message: str
    severity: str = "INFO"  # INFO, WARNING, CRITICAL


class RiskManagerImpl(RiskManager):
    """
    Production-ready risk manager implementation.
    
    Features:
    - Pre-trade risk validation
    - Position limit enforcement
    - Drawdown monitoring
    - Decorator pattern for validation
    """
    
    def __init__(
        self, 
        limits: RiskLimits,
        portfolio: Optional[SimplePortfolio] = None
    ):
        self.limits = limits
        self.portfolio = portfolio
        
        # State tracking
        self._daily_pnl: float = 0.0
        self._peak_equity: float = 0.0
        self._order_timestamps: deque = deque(maxlen=1000)
        self._blocked_symbols: set = set()
        
        self.logger = logging.getLogger("RiskManager")
    
    def check_order(self, event: OrderEvent) -> bool:
        """
        Run all risk checks on order.
        
        Returns True only if ALL checks pass.
        """
        results = self._run_all_checks(event)
        
        # Log results
        for result in results:
            if not result.passed:
                self.logger.warning(
                    f"❌ {result.check_name}: {result.message}"
                )
        
        return all(r.passed for r in results)
    
    def _run_all_checks(self, event: OrderEvent) -> List[RiskCheckResult]:
        """Run all risk checks and return results."""
        checks = [
            self._check_symbol_blocked(event),
            self._check_position_limit(event),
            self._check_notional_limit(event),
            self._check_rate_limit(event),
            self._check_drawdown(),
            self._check_daily_loss()
        ]
        return checks
    
    def _check_symbol_blocked(self, event: OrderEvent) -> RiskCheckResult:
        """Check if symbol is blocked from trading."""
        if event.symbol in self._blocked_symbols:
            return RiskCheckResult(
                passed=False,
                check_name="SYMBOL_BLOCKED",
                message=f"{event.symbol} is blocked from trading",
                severity="CRITICAL"
            )
        return RiskCheckResult(
            passed=True,
            check_name="SYMBOL_BLOCKED",
            message="Symbol clear"
        )
    
    def _check_position_limit(self, event: OrderEvent) -> RiskCheckResult:
        """Check if order would exceed position limits."""
        current_pos = 0
        if self.portfolio:
            current_pos = self.portfolio.current_positions.get(event.symbol, 0)
        
        # Calculate resulting position
        if event.side == OrderSide.BUY:
            new_pos = current_pos + event.quantity
        else:
            new_pos = current_pos - event.quantity
        
        if abs(new_pos) > self.limits.max_position_size:
            return RiskCheckResult(
                passed=False,
                check_name="POSITION_LIMIT",
                message=f"Would exceed max position: {abs(new_pos)} > {self.limits.max_position_size}",
                severity="WARNING"
            )
        return RiskCheckResult(
            passed=True,
            check_name="POSITION_LIMIT",
            message=f"Position OK: {abs(new_pos)} <= {self.limits.max_position_size}"
        )
    
    def _check_notional_limit(self, event: OrderEvent) -> RiskCheckResult:
        """Check if order notional exceeds limit."""
        # Estimate notional (would use market price in production)
        price = event.limit_price or 100.0  # Default assumption
        notional = event.quantity * price
        
        if notional > self.limits.max_notional_order:
            return RiskCheckResult(
                passed=False,
                check_name="NOTIONAL_LIMIT",
                message=f"Notional ${notional:,.2f} > ${self.limits.max_notional_order:,.2f}",
                severity="WARNING"
            )
        return RiskCheckResult(
            passed=True,
            check_name="NOTIONAL_LIMIT",
            message=f"Notional OK: ${notional:,.2f}"
        )
    
    def _check_rate_limit(self, event: OrderEvent) -> RiskCheckResult:
        """Check order rate limit."""
        now = datetime.now()
        cutoff = now - timedelta(minutes=1)
        
        # Count orders in last minute
        self._order_timestamps.append(now)
        recent_orders = sum(1 for ts in self._order_timestamps if ts > cutoff)
        
        if recent_orders > self.limits.max_orders_per_minute:
            return RiskCheckResult(
                passed=False,
                check_name="RATE_LIMIT",
                message=f"Rate limit exceeded: {recent_orders}/min > {self.limits.max_orders_per_minute}/min",
                severity="CRITICAL"
            )
        return RiskCheckResult(
            passed=True,
            check_name="RATE_LIMIT",
            message=f"Rate OK: {recent_orders}/min"
        )
    
    def _check_drawdown(self) -> RiskCheckResult:
        """Check current drawdown against limit."""
        if self.portfolio:
            equity_curve = self.portfolio.equity_curve
            if len(equity_curve) > 0:
                current_equity = equity_curve.iloc[-1]
                self._peak_equity = max(self._peak_equity, current_equity)
                
                if self._peak_equity > 0:
                    drawdown = (self._peak_equity - current_equity) / self._peak_equity
                    
                    if drawdown > self.limits.max_drawdown:
                        return RiskCheckResult(
                            passed=False,
                            check_name="DRAWDOWN",
                            message=f"Drawdown {drawdown:.2%} > {self.limits.max_drawdown:.2%}",
                            severity="CRITICAL"
                        )
        
        return RiskCheckResult(
            passed=True,
            check_name="DRAWDOWN",
            message="Drawdown within limits"
        )
    
    def _check_daily_loss(self) -> RiskCheckResult:
        """Check daily P&L limit."""
        if abs(self._daily_pnl) > self.limits.max_daily_loss:
            return RiskCheckResult(
                passed=False,
                check_name="DAILY_LOSS",
                message=f"Daily loss ${abs(self._daily_pnl):,.2f} > ${self.limits.max_daily_loss:,.2f}",
                severity="CRITICAL"
            )
        return RiskCheckResult(
            passed=True,
            check_name="DAILY_LOSS",
            message=f"Daily P&L: ${self._daily_pnl:,.2f}"
        )
    
    def update_pnl(self, pnl_change: float) -> None:
        """Update daily P&L tracking."""
        self._daily_pnl += pnl_change
    
    def block_symbol(self, symbol: str) -> None:
        """Block a symbol from trading."""
        self._blocked_symbols.add(symbol)
        self.logger.warning(f"🚫 Blocked symbol: {symbol}")
    
    def unblock_symbol(self, symbol: str) -> None:
        """Unblock a symbol."""
        self._blocked_symbols.discard(symbol)
    
    def reset_daily(self) -> None:
        """Reset daily limits (call at start of day)."""
        self._daily_pnl = 0.0
        self._order_timestamps.clear()
    
    def get_risk_metrics(self) -> Dict[str, float]:
        """Get current risk metrics."""
        current_equity = self.portfolio.equity_curve.iloc[-1] if self.portfolio else 0
        drawdown = 0.0
        if self._peak_equity > 0:
            drawdown = (self._peak_equity - current_equity) / self._peak_equity
        
        return {
            'daily_pnl': self._daily_pnl,
            'peak_equity': self._peak_equity,
            'current_equity': current_equity,
            'drawdown': drawdown,
            'orders_last_minute': len(self._order_timestamps),
            'blocked_symbols': len(self._blocked_symbols)
        }


# ============================================================================
# RISK VALIDATION DECORATOR
# ============================================================================

def require_risk_check(risk_manager: RiskManagerImpl):
    """
    Decorator that enforces risk checks before order execution.
    
    Usage:
        @require_risk_check(risk_manager)
        def execute_order(order):
            ...
    """
    def decorator(func):
        def wrapper(order_event: OrderEvent, *args, **kwargs):
            if not risk_manager.check_order(order_event):
                logging.getLogger("RiskDecorator").warning(
                    f"Order rejected by risk manager: {order_event.event_id}"
                )
                return None
            return func(order_event, *args, **kwargs)
        return wrapper
    return decorator


# ============================================================================
# DEMONSTRATE RISK MANAGEMENT
# ============================================================================

print("=" * 60)
print("RISK MANAGEMENT DEMONSTRATION")
print("=" * 60)

# Create risk manager with limits
limits = RiskLimits(
    max_position_size=500,
    max_notional_order=50000.0,
    max_daily_loss=5000.0,
    max_drawdown=0.05
)

portfolio = SimplePortfolio(initial_capital=100000)
risk_mgr = RiskManagerImpl(limits=limits, portfolio=portfolio)

# Test various orders
print("\n🔍 Testing Risk Checks...")

# Normal order - should pass
order_normal = OrderEvent(
    symbol="AAPL",
    side=OrderSide.BUY,
    quantity=100,
    limit_price=150.0
)
result = risk_mgr.check_order(order_normal)
print(f"\n1️⃣ Normal order (100 shares @ $150): {'✅ PASSED' if result else '❌ REJECTED'}")

# Large position - should fail
order_large = OrderEvent(
    symbol="AAPL",
    side=OrderSide.BUY,
    quantity=1000,  # Exceeds 500 limit
    limit_price=150.0
)
result = risk_mgr.check_order(order_large)
print(f"2️⃣ Large position (1000 shares): {'✅ PASSED' if result else '❌ REJECTED'}")

# High notional - should fail
order_notional = OrderEvent(
    symbol="GOOG",
    side=OrderSide.BUY,
    quantity=500,
    limit_price=150.0  # 500 * 150 = 75000 > 50000 limit
)
result = risk_mgr.check_order(order_notional)
print(f"3️⃣ High notional ($75,000): {'✅ PASSED' if result else '❌ REJECTED'}")

# Block a symbol
risk_mgr.block_symbol("TSLA")
order_blocked = OrderEvent(
    symbol="TSLA",
    side=OrderSide.BUY,
    quantity=100,
    limit_price=200.0
)
result = risk_mgr.check_order(order_blocked)
print(f"4️⃣ Blocked symbol (TSLA): {'✅ PASSED' if result else '❌ REJECTED'}")

# Show risk metrics
print("\n📊 Risk Metrics:")
metrics = risk_mgr.get_risk_metrics()
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"   {key}: {value:.2f}")
    else:
        print(f"   {key}: {value}")

## 7. Backtesting Engine Architecture

The backtesting engine orchestrates all components to simulate historical trading. It's the integration point that ties everything together.

```
Backtest Engine Event Loop:
───────────────────────────────────────────────────────────────
┌──────────────────────────────────────────────────────────────┐
│                    BACKTEST ENGINE                            │
├──────────────────────────────────────────────────────────────┤
│                                                               │
│  while data_handler.has_more_data:                           │
│      │                                                        │
│      ├─▶ 1. data_handler.update_bars()                       │
│      │      └─▶ Publishes MarketDataEvent                    │
│      │                                                        │
│      ├─▶ 2. strategy.calculate_signals(market_event)         │
│      │      └─▶ Publishes SignalEvent (if generated)         │
│      │                                                        │
│      ├─▶ 3. portfolio.update_signal(signal_event)            │
│      │      └─▶ Publishes OrderEvent                         │
│      │                                                        │
│      ├─▶ 4. risk_manager.check_order(order_event)            │
│      │      └─▶ Pass/Reject order                            │
│      │                                                        │
│      ├─▶ 5. execution_handler.execute_order(order_event)     │
│      │      └─▶ Publishes FillEvent                          │
│      │                                                        │
│      └─▶ 6. portfolio.update_fill(fill_event)                │
│             └─▶ Updates positions and P&L                    │
│                                                               │
└──────────────────────────────────────────────────────────────┘
```

In [ ]:
# ============================================================================
# BACKTESTING ENGINE
# ============================================================================

@dataclass
class BacktestConfig:
    """Configuration for backtest runs."""
    initial_capital: float = 100000.0
    commission_per_share: float = 0.01
    slippage_pct: float = 0.001
    start_date: Optional[datetime] = None
    end_date: Optional[datetime] = None


@dataclass
class BacktestResults:
    """Results container for backtest output."""
    total_return: float
    sharpe_ratio: float
    max_drawdown: float
    win_rate: float
    total_trades: int
    equity_curve: pd.Series
    trade_log: List[Dict]
    
    def summary(self) -> str:
        """Generate results summary."""
        return f"""
╔══════════════════════════════════════════════════════════════╗
║                    BACKTEST RESULTS                           ║
╠══════════════════════════════════════════════════════════════╣
║  Total Return:    {self.total_return:>10.2%}                          ║
║  Sharpe Ratio:    {self.sharpe_ratio:>10.2f}                          ║
║  Max Drawdown:    {self.max_drawdown:>10.2%}                          ║
║  Win Rate:        {self.win_rate:>10.2%}                          ║
║  Total Trades:    {self.total_trades:>10d}                          ║
╚══════════════════════════════════════════════════════════════╝
"""


class BacktestEngine:
    """
    Event-driven backtesting engine.
    
    Orchestrates all components through the event bus to simulate
    historical trading with realistic execution.
    """
    
    def __init__(
        self,
        data_handler: DataHandler,
        strategy: Strategy,
        portfolio: Portfolio,
        execution_handler: ExecutionHandler,
        risk_manager: RiskManager,
        event_bus: EventBus,
        config: BacktestConfig
    ):
        self.data_handler = data_handler
        self.strategy = strategy
        self.portfolio = portfolio
        self.execution_handler = execution_handler
        self.risk_manager = risk_manager
        self.event_bus = event_bus
        self.config = config
        
        self._trade_log: List[Dict] = []
        self._current_bar: int = 0
        self.logger = logging.getLogger("BacktestEngine")
        
        # Wire up event handlers
        self._setup_event_handlers()
    
    def _setup_event_handlers(self) -> None:
        """Subscribe components to event types."""
        # Market data -> Strategy
        self.event_bus.subscribe(
            EventType.MARKET_DATA,
            self._on_market_data
        )
        
        # Signal -> Portfolio
        self.event_bus.subscribe(
            EventType.SIGNAL,
            self._on_signal
        )
        
        # Order -> Execution (through risk check)
        self.event_bus.subscribe(
            EventType.ORDER,
            self._on_order
        )
        
        # Fill -> Portfolio
        self.event_bus.subscribe(
            EventType.FILL,
            self._on_fill
        )
    
    def _on_market_data(self, event: MarketDataEvent) -> None:
        """Handle market data event."""
        # Update execution handler with latest price
        if isinstance(self.execution_handler, SimulatedExecutionHandler):
            self.execution_handler.update_price(event.symbol, event.close_price)
        
        # Generate signals
        signal = self.strategy.calculate_signals(event)
        if signal:
            self.event_bus.publish(signal)
    
    def _on_signal(self, event: SignalEvent) -> None:
        """Handle signal event."""
        # Convert signal to order
        order = self.portfolio.update_signal(event)
        if order:
            self.event_bus.publish(order)
    
    def _on_order(self, event: OrderEvent) -> None:
        """Handle order event with risk check."""
        # Risk check
        if not self.risk_manager.check_order(event):
            self.logger.warning(f"Order rejected by risk manager: {event.event_id}")
            return
        
        # Execute order
        fill = self.execution_handler.execute_order(event)
        if fill:
            self.event_bus.publish(fill)
    
    def _on_fill(self, event: FillEvent) -> None:
        """Handle fill event."""
        self.portfolio.update_fill(event)
        
        # Log trade
        self._trade_log.append({
            'timestamp': event.timestamp,
            'symbol': event.symbol,
            'side': event.side.value,
            'quantity': event.quantity,
            'price': event.fill_price,
            'commission': event.commission
        })
    
    def run(self) -> BacktestResults:
        """
        Execute the backtest.
        
        Returns:
            BacktestResults with performance metrics
        """
        self.logger.info("🚀 Starting backtest...")
        start_time = time.time()
        
        # Main event loop
        bar_count = 0
        while self.data_handler.continue_backtest:
            # Push next bar (publishes MarketDataEvent)
            self.data_handler.update_bars()
            
            # Process all events in queue
            self.event_bus.process_all()
            
            bar_count += 1
            if bar_count % 100 == 0:
                self.logger.info(f"Processed {bar_count} bars...")
        
        elapsed = time.time() - start_time
        self.logger.info(f"✅ Backtest complete in {elapsed:.2f}s ({bar_count} bars)")
        
        # Calculate results
        return self._calculate_results()
    
    def _calculate_results(self) -> BacktestResults:
        """Calculate backtest performance metrics."""
        equity_curve = self.portfolio.equity_curve
        
        if len(equity_curve) < 2:
            return BacktestResults(
                total_return=0.0,
                sharpe_ratio=0.0,
                max_drawdown=0.0,
                win_rate=0.0,
                total_trades=0,
                equity_curve=equity_curve,
                trade_log=self._trade_log
            )
        
        # Total return
        total_return = (equity_curve.iloc[-1] / equity_curve.iloc[0]) - 1
        
        # Daily returns and Sharpe ratio
        returns = equity_curve.pct_change().dropna()
        if len(returns) > 0 and returns.std() > 0:
            sharpe_ratio = np.sqrt(252) * returns.mean() / returns.std()
        else:
            sharpe_ratio = 0.0
        
        # Max drawdown
        cummax = equity_curve.cummax()
        drawdown = (cummax - equity_curve) / cummax
        max_drawdown = drawdown.max()
        
        # Win rate (from trades)
        if len(self._trade_log) >= 2:
            # Pair up trades to calculate P&L
            pnls = []
            for i in range(0, len(self._trade_log) - 1, 2):
                trade1 = self._trade_log[i]
                trade2 = self._trade_log[i + 1]
                
                if trade1['side'] == 'BUY':
                    pnl = (trade2['price'] - trade1['price']) * trade1['quantity']
                else:
                    pnl = (trade1['price'] - trade2['price']) * trade1['quantity']
                pnls.append(pnl)
            
            win_rate = len([p for p in pnls if p > 0]) / len(pnls) if pnls else 0
        else:
            win_rate = 0.0
        
        return BacktestResults(
            total_return=total_return,
            sharpe_ratio=sharpe_ratio,
            max_drawdown=max_drawdown,
            win_rate=win_rate,
            total_trades=len(self._trade_log),
            equity_curve=equity_curve,
            trade_log=self._trade_log
        )


# ============================================================================
# RUN COMPLETE BACKTEST
# ============================================================================

print("=" * 60)
print("BACKTEST ENGINE DEMONSTRATION")
print("=" * 60)

# Generate more data for meaningful backtest
np.random.seed(42)
n_days = 252  # 1 year
dates = pd.date_range('2024-01-01', periods=n_days, freq='D')

# Generate trending price with noise
trend = np.linspace(0, 0.2, n_days)  # 20% uptrend
noise = np.random.randn(n_days) * 0.02
prices = 100 * np.cumprod(1 + trend/n_days + noise)

backtest_data = pd.DataFrame({
    'symbol': 'AAPL',
    'open': prices * (1 + np.random.randn(n_days) * 0.005),
    'high': prices * (1 + np.abs(np.random.randn(n_days)) * 0.01),
    'low': prices * (1 - np.abs(np.random.randn(n_days)) * 0.01),
    'close': prices,
    'volume': np.random.randint(1000000, 5000000, n_days)
}, index=dates)

# Create components
config = BacktestConfig(
    initial_capital=100000.0,
    commission_per_share=0.01,
    slippage_pct=0.001
)

event_bus = EventBus()
data_handler = HistoricalDataHandler(backtest_data, event_bus)
strategy = MomentumStrategy(short_window=10, long_window=30)
portfolio = SimplePortfolio(
    initial_capital=config.initial_capital,
    position_size=100
)
execution_handler = SimulatedExecutionHandler(
    slippage_pct=config.slippage_pct,
    commission_per_share=config.commission_per_share
)
risk_limits = RiskLimits(max_position_size=500)
risk_manager = RiskManagerImpl(limits=risk_limits, portfolio=portfolio)

# Create and run backtest engine
engine = BacktestEngine(
    data_handler=data_handler,
    strategy=strategy,
    portfolio=portfolio,
    execution_handler=execution_handler,
    risk_manager=risk_manager,
    event_bus=event_bus,
    config=config
)

print("\n🏃 Running backtest on 1 year of daily data...")
results = engine.run()

# Display results
print(results.summary())

# Show some trades
if results.trade_log:
    print("📋 Sample Trades:")
    for trade in results.trade_log[:6]:
        print(f"   {trade['timestamp'].date()} | {trade['side']:4s} | "
              f"{trade['quantity']:4d} @ ${trade['price']:.2f}")

## 8. Live Trading System State Machine

Production trading systems require careful state management. A state machine ensures valid transitions and proper handling of edge cases.

```
Trading System State Machine:
───────────────────────────────────────────────────────────────

     ┌─────────────────────────────────────────────────────┐
     │                                                      │
     │    ┌──────────────┐                                 │
     │    │ INITIALIZING │ ──start()──▶ ┌──────────────┐   │
     │    └──────────────┘              │  CONNECTING  │   │
     │           ▲                      └───────┬──────┘   │
     │           │                              │          │
     │      shutdown()                    connected()      │
     │           │                              │          │
     │    ┌──────┴───────┐                      ▼          │
     │    │   SHUTDOWN   │◀──── shutdown() ┌────────┐      │
     │    └──────────────┘                 │RUNNING │      │
     │           ▲                         └───┬────┘      │
     │           │                             │           │
     │      shutdown()                     pause()         │
     │           │                             │           │
     │    ┌──────┴───────┐    resume()         ▼          │
     │    │    ERROR     │◀─────────────┌──────────┐      │
     │    └──────────────┘              │  PAUSED  │      │
     │                                   └──────────┘      │
     │                                                      │
     └─────────────────────────────────────────────────────┘
```

In [ ]:
# ============================================================================
# LIVE TRADING SYSTEM STATE MACHINE
# ============================================================================

class SystemState(Enum):
    """Trading system states."""
    INITIALIZING = "INITIALIZING"
    CONNECTING = "CONNECTING"
    RUNNING = "RUNNING"
    PAUSED = "PAUSED"
    SHUTDOWN = "SHUTDOWN"
    ERROR = "ERROR"


@dataclass
class StateTransition:
    """Record of a state transition."""
    from_state: SystemState
    to_state: SystemState
    timestamp: datetime
    reason: str = ""


class InvalidStateTransition(Exception):
    """Raised when an invalid state transition is attempted."""
    pass


class TradingSystemStateMachine:
    """
    State machine for live trading system lifecycle.
    
    Ensures valid state transitions and provides hooks for
    actions on state entry/exit.
    """
    
    # Valid state transitions: {from_state: [valid_to_states]}
    VALID_TRANSITIONS = {
        SystemState.INITIALIZING: [SystemState.CONNECTING, SystemState.SHUTDOWN],
        SystemState.CONNECTING: [SystemState.RUNNING, SystemState.ERROR, SystemState.SHUTDOWN],
        SystemState.RUNNING: [SystemState.PAUSED, SystemState.SHUTDOWN, SystemState.ERROR],
        SystemState.PAUSED: [SystemState.RUNNING, SystemState.SHUTDOWN],
        SystemState.ERROR: [SystemState.SHUTDOWN, SystemState.INITIALIZING],
        SystemState.SHUTDOWN: [],  # Terminal state
    }
    
    def __init__(self):
        self._state = SystemState.INITIALIZING
        self._state_history: List[StateTransition] = []
        self._entry_actions: Dict[SystemState, List[Callable]] = {s: [] for s in SystemState}
        self._exit_actions: Dict[SystemState, List[Callable]] = {s: [] for s in SystemState}
        self._lock = Lock()
        self.logger = logging.getLogger("StateMachine")
        
        # Record initial state
        self._state_history.append(StateTransition(
            from_state=None,
            to_state=SystemState.INITIALIZING,
            timestamp=datetime.now(),
            reason="System initialization"
        ))
    
    @property
    def state(self) -> SystemState:
        """Current system state."""
        return self._state
    
    @property
    def is_running(self) -> bool:
        """Check if system is in running state."""
        return self._state == SystemState.RUNNING
    
    @property
    def is_active(self) -> bool:
        """Check if system is active (running or paused)."""
        return self._state in [SystemState.RUNNING, SystemState.PAUSED]
    
    def can_transition_to(self, target: SystemState) -> bool:
        """Check if transition to target state is valid."""
        return target in self.VALID_TRANSITIONS.get(self._state, [])
    
    def transition_to(self, target: SystemState, reason: str = "") -> bool:
        """
        Attempt state transition.
        
        Args:
            target: Target state
            reason: Reason for transition (for logging)
            
        Returns:
            True if transition successful
            
        Raises:
            InvalidStateTransition if transition not allowed
        """
        with self._lock:
            if not self.can_transition_to(target):
                raise InvalidStateTransition(
                    f"Cannot transition from {self._state.value} to {target.value}"
                )
            
            old_state = self._state
            
            # Execute exit actions
            for action in self._exit_actions[old_state]:
                try:
                    action(old_state, target)
                except Exception as e:
                    self.logger.error(f"Exit action failed: {e}")
            
            # Transition
            self._state = target
            
            # Record transition
            self._state_history.append(StateTransition(
                from_state=old_state,
                to_state=target,
                timestamp=datetime.now(),
                reason=reason
            ))
            
            self.logger.info(
                f"State: {old_state.value} → {target.value} ({reason})"
            )
            
            # Execute entry actions
            for action in self._entry_actions[target]:
                try:
                    action(old_state, target)
                except Exception as e:
                    self.logger.error(f"Entry action failed: {e}")
            
            return True
    
    def on_entry(self, state: SystemState, action: Callable) -> None:
        """Register action to run when entering a state."""
        self._entry_actions[state].append(action)
    
    def on_exit(self, state: SystemState, action: Callable) -> None:
        """Register action to run when exiting a state."""
        self._exit_actions[state].append(action)
    
    # Convenience methods for common transitions
    def start(self) -> bool:
        """Start the system (INITIALIZING -> CONNECTING)."""
        return self.transition_to(SystemState.CONNECTING, "Starting system")
    
    def connected(self) -> bool:
        """Mark as connected (CONNECTING -> RUNNING)."""
        return self.transition_to(SystemState.RUNNING, "Connection established")
    
    def pause(self) -> bool:
        """Pause trading (RUNNING -> PAUSED)."""
        return self.transition_to(SystemState.PAUSED, "Trading paused")
    
    def resume(self) -> bool:
        """Resume trading (PAUSED -> RUNNING)."""
        return self.transition_to(SystemState.RUNNING, "Trading resumed")
    
    def shutdown(self) -> bool:
        """Shutdown system."""
        return self.transition_to(SystemState.SHUTDOWN, "System shutdown")
    
    def error(self, message: str = "") -> bool:
        """Enter error state."""
        return self.transition_to(SystemState.ERROR, f"Error: {message}")
    
    def get_uptime(self) -> timedelta:
        """Get time since system started."""
        if self._state_history:
            return datetime.now() - self._state_history[0].timestamp
        return timedelta(0)
    
    def get_state_history(self) -> List[StateTransition]:
        """Get full state transition history."""
        return self._state_history.copy()


# ============================================================================
# LIVE TRADING SYSTEM WITH HEALTH MONITORING
# ============================================================================

@dataclass
class HealthCheck:
    """Health check result."""
    component: str
    status: str  # "healthy", "degraded", "unhealthy"
    latency_ms: float
    last_check: datetime
    message: str = ""


class LiveTradingSystem:
    """
    Production live trading system with health monitoring.
    
    Combines:
    - State machine for lifecycle management
    - Health monitoring for all components
    - Graceful degradation and recovery
    """
    
    def __init__(
        self,
        strategy: Strategy,
        portfolio: Portfolio,
        risk_manager: RiskManager,
        event_bus: EventBus
    ):
        self.strategy = strategy
        self.portfolio = portfolio
        self.risk_manager = risk_manager
        self.event_bus = event_bus
        
        # State machine
        self.state_machine = TradingSystemStateMachine()
        self._setup_state_actions()
        
        # Health monitoring
        self._health_checks: Dict[str, HealthCheck] = {}
        self._last_heartbeat: datetime = datetime.now()
        
        self.logger = logging.getLogger("LiveTradingSystem")
    
    def _setup_state_actions(self) -> None:
        """Configure state entry/exit actions."""
        # On entering RUNNING
        self.state_machine.on_entry(
            SystemState.RUNNING,
            lambda old, new: self.logger.info("🟢 System now RUNNING")
        )
        
        # On entering PAUSED
        self.state_machine.on_entry(
            SystemState.PAUSED,
            lambda old, new: self.logger.info("⏸️ System PAUSED")
        )
        
        # On entering SHUTDOWN
        self.state_machine.on_entry(
            SystemState.SHUTDOWN,
            lambda old, new: self._cleanup()
        )
        
        # On entering ERROR
        self.state_machine.on_entry(
            SystemState.ERROR,
            lambda old, new: self._handle_error()
        )
    
    def _cleanup(self) -> None:
        """Cleanup on shutdown."""
        self.logger.info("🛑 Cleaning up resources...")
        # Cancel open orders, close positions, etc.
    
    def _handle_error(self) -> None:
        """Handle error state."""
        self.logger.error("❌ System in ERROR state")
        # Alert, save state, attempt recovery, etc.
    
    def check_health(self) -> Dict[str, HealthCheck]:
        """Run health checks on all components."""
        now = datetime.now()
        
        # Check event bus
        self._health_checks['event_bus'] = HealthCheck(
            component='event_bus',
            status='healthy' if self.event_bus._events_processed >= 0 else 'unhealthy',
            latency_ms=0.1,
            last_check=now
        )
        
        # Check strategy
        self._health_checks['strategy'] = HealthCheck(
            component='strategy',
            status='healthy',
            latency_ms=0.5,
            last_check=now
        )
        
        # Check risk manager
        risk_metrics = self.risk_manager.get_risk_metrics()
        drawdown = risk_metrics.get('drawdown', 0)
        status = 'healthy' if drawdown < 0.05 else ('degraded' if drawdown < 0.1 else 'unhealthy')
        self._health_checks['risk_manager'] = HealthCheck(
            component='risk_manager',
            status=status,
            latency_ms=0.2,
            last_check=now,
            message=f"Drawdown: {drawdown:.2%}"
        )
        
        return self._health_checks
    
    def get_system_status(self) -> Dict[str, Any]:
        """Get comprehensive system status."""
        health = self.check_health()
        
        unhealthy_count = sum(1 for h in health.values() if h.status == 'unhealthy')
        degraded_count = sum(1 for h in health.values() if h.status == 'degraded')
        
        overall = 'healthy'
        if unhealthy_count > 0:
            overall = 'unhealthy'
        elif degraded_count > 0:
            overall = 'degraded'
        
        return {
            'state': self.state_machine.state.value,
            'overall_health': overall,
            'uptime': str(self.state_machine.get_uptime()),
            'components': {
                name: {
                    'status': h.status,
                    'latency_ms': h.latency_ms,
                    'message': h.message
                }
                for name, h in health.items()
            }
        }
    
    def start(self) -> bool:
        """Start the trading system."""
        try:
            self.state_machine.start()
            
            # Simulate connection
            time.sleep(0.1)
            self.state_machine.connected()
            
            return True
        except InvalidStateTransition as e:
            self.logger.error(f"Failed to start: {e}")
            return False
    
    def stop(self) -> bool:
        """Stop the trading system."""
        try:
            self.state_machine.shutdown()
            return True
        except InvalidStateTransition as e:
            self.logger.error(f"Failed to stop: {e}")
            return False


# ============================================================================
# DEMONSTRATE LIVE SYSTEM
# ============================================================================

print("=" * 60)
print("LIVE TRADING SYSTEM DEMONSTRATION")
print("=" * 60)

# Create components
live_event_bus = EventBus()
live_strategy = MomentumStrategy(short_window=5, long_window=20)
live_portfolio = SimplePortfolio(initial_capital=100000)
live_risk_limits = RiskLimits()
live_risk_mgr = RiskManagerImpl(limits=live_risk_limits, portfolio=live_portfolio)

# Create live system
live_system = LiveTradingSystem(
    strategy=live_strategy,
    portfolio=live_portfolio,
    risk_manager=live_risk_mgr,
    event_bus=live_event_bus
)

# Show initial state
print(f"\n📊 Initial State: {live_system.state_machine.state.value}")

# Start system
print("\n🚀 Starting system...")
live_system.start()

# Check status
status = live_system.get_system_status()
print(f"\n📋 System Status:")
print(f"   State: {status['state']}")
print(f"   Health: {status['overall_health']}")
print(f"   Uptime: {status['uptime']}")

# Simulate pause/resume
print("\n⏸️ Pausing system...")
live_system.state_machine.pause()
print(f"   State: {live_system.state_machine.state.value}")

print("\n▶️ Resuming system...")
live_system.state_machine.resume()
print(f"   State: {live_system.state_machine.state.value}")

# Show state history
print("\n📜 State History:")
for transition in live_system.state_machine.get_state_history():
    print(f"   {transition.to_state.value}: {transition.reason}")

# Shutdown
print("\n🛑 Shutting down...")
live_system.stop()
print(f"   Final State: {live_system.state_machine.state.value}")

## 9. Summary: Architecture Patterns Reference

### Key Design Patterns Used

| Pattern | Purpose | Example in Code |
|---------|---------|-----------------|
| **Event-Driven** | Decouple components through events | `EventBus`, `Event` classes |
| **Strategy** | Interchangeable algorithms | `Strategy` ABC, `MomentumStrategy` |
| **Observer/Pub-Sub** | Multiple subscribers to events | `EventBus.subscribe()` |
| **State Machine** | Lifecycle management | `TradingSystemStateMachine` |
| **Factory** | Create objects without specifying class | `OMS.create_order()` |
| **Decorator** | Add behavior to functions | `@require_risk_check` |
| **Pipeline** | Chain data transformations | `DataFetcher → Cleaner → Features` |
| **Template Method** | Define algorithm skeleton | ABC methods with concrete implementations |

### Interview Tips

**Common Questions:**
1. *"How would you design a trading system?"* → Start with event-driven architecture diagram
2. *"How do you handle failures?"* → State machine + health monitoring + graceful degradation
3. *"How do you ensure low latency?"* → Generator patterns, in-memory processing, efficient data structures
4. *"How do you test trading systems?"* → Event replay, mock components, backtesting engine

**Key Principles to Mention:**
- Separation of concerns
- Single responsibility principle
- Dependency injection
- Event sourcing for auditability
- Idempotent operations for reliability

In [ ]:
# ============================================================================
# COMPLETE ARCHITECTURE DIAGRAM
# ============================================================================

architecture_diagram = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                    COMPLETE TRADING SYSTEM ARCHITECTURE                       ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                        EXTERNAL SYSTEMS                                  │ ║
║  │  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐  ┌─────────────┐ │ ║
║  │  │ Market Data  │  │   Exchange   │  │   Broker    │  │   News/Alt  │ │ ║
║  │  │   Vendors    │  │    APIs      │  │    APIs     │  │    Data     │ │ ║
║  │  └──────┬───────┘  └──────┬───────┘  └──────┬───────┘  └──────┬──────┘ │ ║
║  └─────────┼─────────────────┼─────────────────┼─────────────────┼────────┘ ║
║            │                 │                 │                 │          ║
║            ▼                 ▼                 ▼                 ▼          ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                         DATA LAYER                                       │ ║
║  │  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐  ┌─────────────┐ │ ║
║  │  │ DataFetcher  │──│ DataCleaner  │──│FeatureEngine │──│  DataStore  │ │ ║
║  │  └──────────────┘  └──────────────┘  └──────────────┘  └─────────────┘ │ ║
║  └──────────────────────────────┬──────────────────────────────────────────┘ ║
║                                 │                                            ║
║                                 ▼                                            ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                         EVENT BUS (Central Nervous System)               │ ║
║  │                                                                          │ ║
║  │  MarketDataEvent ─▶ SignalEvent ─▶ OrderEvent ─▶ FillEvent              │ ║
║  │                                                                          │ ║
║  └──────────────────────────────┬──────────────────────────────────────────┘ ║
║                                 │                                            ║
║     ┌───────────────────────────┼───────────────────────────┐               ║
║     │                           │                           │               ║
║     ▼                           ▼                           ▼               ║
║  ┌──────────────┐         ┌──────────────┐         ┌──────────────┐        ║
║  │   STRATEGY   │         │  PORTFOLIO   │         │   EXECUTION  │        ║
║  │    ENGINE    │         │   MANAGER    │         │   HANDLER    │        ║
║  │              │         │              │         │              │        ║
║  │ • Signal Gen │         │ • Positions  │         │ • Order Mgmt │        ║
║  │ • Alpha Calc │         │ • P&L Track  │         │ • Fill Sim   │        ║
║  │ • ML Models  │         │ • Allocation │         │ • Slippage   │        ║
║  └──────┬───────┘         └──────┬───────┘         └──────┬───────┘        ║
║         │                        │                        │                 ║
║         └────────────────────────┼────────────────────────┘                 ║
║                                  │                                          ║
║                                  ▼                                          ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                         RISK MANAGEMENT                                  │ ║
║  │                                                                          │ ║
║  │  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐    │ ║
║  │  │  Position   │  │   P&L       │  │  Drawdown   │  │    VaR      │    │ ║
║  │  │   Limits    │  │   Limits    │  │   Monitor   │  │   Check     │    │ ║
║  │  └─────────────┘  └─────────────┘  └─────────────┘  └─────────────┘    │ ║
║  │                                                                          │ ║
║  └─────────────────────────────────────────────────────────────────────────┘ ║
║                                                                               ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                    SYSTEM MANAGEMENT LAYER                               │ ║
║  │                                                                          │ ║
║  │  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐    │ ║
║  │  │State Machine│  │   Health    │  │   Logging   │  │   Alerting  │    │ ║
║  │  │ (Lifecycle) │  │  Monitoring │  │  & Metrics  │  │  & Recovery │    │ ║
║  │  └─────────────┘  └─────────────┘  └─────────────┘  └─────────────┘    │ ║
║  │                                                                          │ ║
║  └─────────────────────────────────────────────────────────────────────────┘ ║
║                                                                               ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(architecture_diagram)

# Summary statistics
print("\n" + "=" * 60)
print("📚 NOTEBOOK SUMMARY")
print("=" * 60)

print("""
Components Implemented:
  ✅ Event System (Events, EventBus, Pub/Sub)
  ✅ Abstract Component Interfaces (ABC pattern)
  ✅ Data Pipeline (Generator-based streaming)
  ✅ Order Management System (Order lifecycle)
  ✅ Risk Management (Pre-trade checks, limits)
  ✅ Backtesting Engine (Event-driven simulation)
  ✅ Live Trading State Machine (Lifecycle management)

Design Patterns Used:
  • Event-Driven Architecture
  • Strategy Pattern
  • Observer/Pub-Sub Pattern
  • State Machine Pattern
  • Factory Pattern
  • Decorator Pattern
  • Pipeline Pattern
  • Template Method Pattern

Next Steps for Day 2:
  → Database Design (Time-series, Redis, PostgreSQL)
  → API Design (REST, WebSocket, FIX protocol)
  → Containerization (Docker, Kubernetes)
  → Monitoring (Prometheus, Grafana)
""")